# 필수 모듈 가져오기

In [7]:
from numpy import int16
import requests
import json
from playsound import playsound
from scipy.io.wavfile import write
import sounddevice
import multiprocessing
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver

# 음성 녹음

In [58]:
fs=16000
second=5
print("recording....")
record_voice=sounddevice.rec(int(second * fs), samplerate=fs, channels=1, dtype=int16)
sounddevice.wait()
write('temp.wav', fs, record_voice)

recording....


# STT 서비스 요청

In [59]:
url = "https://kakaoi-newtone-openapi.kakao.com/v1/recognize"
key = "931127ffaae1b169d808ee442f53e268"
headers = {
    "Content-Type": "application/octet-stream",
    "Authorization": "KakaoAK " + key,
}
with open('temp.wav', 'rb') as fp:
    audio = fp.read()
res = requests.post(url, headers=headers, data=audio)

# 요청 결과 출력

In [60]:
json_string = res.text[res.text.index('{"type":"finalResult"'):res.text.rindex('}')+1]
result = json.loads(json_string)['value']
print(result)

유튜브 열어 줘


# 키워드 비교

In [61]:
keyword = ''
result = result.replace(" ","")
if "몇시" in result:
    keyword = 'CURRENT_TIME'
elif "뉴스" in result:
    keyword = 'NEWS'
elif "열어줘" in result or "사이트" in result:
    keyword = 'SITE'
else:
    keyword = 'UNKNOWN'

# 키워드 출력

In [62]:
print(keyword)

SITE


# 키워드에 따라 처리

In [63]:
data = ''
siteList = {'네이버':'http://naver.com', '다음':'http://daum.net', '유튜브': 'http://youtube.com'}
siteName = ''
siteUrl = ''

if keyword == 'CURRENT_TIME':
    now = datetime.today().strftime('%H:%M')
    data = f'<speak>현재 시각은<break time="150ms"/>\
              <say-as interpret-as="time" format="hm24"> {now} </say-as> 입니다</speak>'
elif keyword == 'NEWS':
    newsList = []
    url = 'https://www.yonhapnewstv.co.kr/news/headline'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}
    res = requests.get(url, headers=headers)
    if (res.status_code != 200):
        print(f'Request 요청이 실패하였습니다.(code={res.status_code})')
        exit(1)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    elem = soup.findAll('strong', class_='title')
    data = '<speak>오늘의 뉴스를 알려드리겠습니다.'
    for n, i in enumerate(elem):
        news = i.text.replace('\n', '').replace('  ', '')
        data += f'{n+1}번: {news}<break time="500ms"/>'
    data += '</speak>'
elif keyword == 'SITE':
    for site, url in siteList.items():
        if site in result:
            data = f'<speak>{site} 페이지를 열겠습니다.</speak>'
            siteName = site
            siteUrl = url
else:
    data = f'<speak>무슨 말인지 잘 모르겠어요</speak>'
data = data.encode('utf-8').decode('latin1')    

# TTS 서비스 요청

In [64]:
url = "https://kakaoi-newtone-openapi.kakao.com/v1/synthesize"
key = "931127ffaae1b169d808ee442f53e268"
headers = {
    "Content-Type": "application/xml",
    "Authorization": "KakaoAK " + key,
}
res = requests.post(url, headers=headers, data=data)
if (res.status_code != 200):
    print(f'TTS 요청이 실패하였습니다.(code={res.status_code})')
else:
    with open('result.mp3', "wb") as f:
        f.write(res.content)

# TTS 서비스 결과 출력

In [66]:
p = multiprocessing.Process(target=playsound, args=('result.mp3',))
p.start()
input("press ENTER to stop playback")
p.terminate()

press ENTER to stop playback


# 웹 브라우저 열기

In [67]:
if keyword == "SITE" and siteUrl:
    service = webdriver.chrome.service.Service("C:/Users/oralcoder/Downloads/chromedriver_win32/chromedriver.exe")
    browser = webdriver.Chrome(service=service)
    browser.get(siteUrl)

In [71]:
import os
os.startfile("C:/Program Files (x86)/HNC/Office 2018/HOffice100/Bin/Hwp.exe")